In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain.document_loaders import PyPDFLoader
import pdfplumber
import pandas as pd
import os
from datetime import datetime
from langchain.text_splitter import RecursiveCharacterTextSplitter
import faiss
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough


model = ChatOpenAI(model="gpt-4o-mini")

loader = PyPDFLoader("ai_industry.pdf")
docs = loader.load()

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(docs)

with pdfplumber.open("ai_industry.pdf") as pdf:
    all_tables = []  
    
    for page_num, page in enumerate(pdf.pages):
        tables = page.extract_tables()  
        
        for table in tables:
            df = pd.DataFrame(table[1:], columns=table[0])
            all_tables.append((page_num, df)) 

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# 프롬프트 파일 로드
prompts = {}
prompt_folder = "C:/Users/hispa/promptengineering/Prompts"
for filename in os.listdir(prompt_folder):
    if filename.endswith(".txt"):
        with open(os.path.join(prompt_folder, filename), "r", encoding="utf-8") as file:
            prompts[filename] = file.read()

class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output

class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  
        # context의 각 문서를 문자열로 결합
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        # 전체 프롬프트를 단일 문자열로 생성
        full_prompt = f"Context: {context_text}\n\nQuestion: {inputs['question']}"
        return full_prompt

# 결과 저장 디렉토리 
result_folder = "C:/Users/hispa/promptengineering/Results"
os.makedirs(result_folder, exist_ok=True)

rag_chain_debug = DebugPassThrough() | ContextToText() | model

In [8]:
while True: 
    print("========================")
    query = input("질문을 입력하세요 (종료하려면 'exit' 입력): ")
    if query == "exit":
        break
    

    for i, (prompt_name, prompt_text) in enumerate(prompts.items(), start=1):
        print(f"\nPrompt {i}: {prompt_name}")
        
        context_documents = retriever.get_relevant_documents(query)
        
        inputs = {"context": context_documents, "question": prompt_text + "\n" + query}

        response = rag_chain_debug.invoke(inputs)

        print(f"Response for {prompt_name}:")
        print(response.content)
        
        timestamp = int(datetime.now().timestamp())
        result_filename = f"{prompt_name.replace('.txt', '')}_result_{timestamp}.txt"
        with open(os.path.join(result_folder, result_filename), "w", encoding="utf-8") as result_file:
            result_file.write(response.content)


질문을 입력하세요 (종료하려면 'exit' 입력):  가트너가 뭐야?



Prompt 1: prompt1.txt
Debug Output: {'context': [Document(metadata={'source': 'ai_industry.pdf', 'page': 18}, page_content='£AI 에이전트 활성화를 위해 향상된 메모리 성능을 갖춘 대체 아키텍처 채택 필요n이스라엘의 AI 스타트업 AI21의 오리 고센(Ori Goshen) CEO가 AI 에이전트를 활성화하려면 트랜스포머(Transformer)* 이외의 새로운 아키텍처**가 필요하다고 주장* 문장 속 단어와 같은 순차 데이터 내의 관계를 추적해 맥락과 의미를 학습하는 신경망 ** AI 시스템이 데이터를 처리하고 학습하기 위한 신경망의 전체적인 구조와 설계 방식을 의미 ∙트랜스포머는 현재 AI 모델 개발에서 가장 많이 사용되는 아키텍처이지만, 다중 에이전트 생태계 조성 측면에서는 한계를 내포∙트랜스포머 아키텍처는 처리하는 컨텍스트가 길수록 속도가 느리고 연산 비용이 많이 드는데, AI 에이전트는 LLM을 여러 차례 호출해야 하고 각 단계에서 광범위한 컨텍스트를 사용하는 경우가 많아 처리 과정에서 지연이 발생n고센 CEO는 ‘맘바(Mamba)’와 ‘잠바(Jamba)’와 같은 대체 아키텍처를 활용하면 AI 에이전트를 더 효율적이고 저렴하게 만들 수 있다고 강조∙카네기멜론⼤와 프린스턴⼤ 연구진이 개발한 맘바는 트랜스포머 모델의 핵심인 어텐션(Attention)* 메커니즘 대신 데이터를 우선순위에 따라 정리하고 입력에 가중치를 부여해 메모리 사용을 최적화* 입력된 데이터 간 연관성을 파악해 상호작용을 계산하는 메커니즘 ∙미스트랄이 2024년 7월 ‘코드스트랄(Codestral) 맘바 7B’를, UAE의 AI 기업 팔콘(Falcon)이 8월 ‘팔콘 맘바 7B’를 출시하는 등, 최근 오픈소스 AI 개발자 사이에서 맘바의 인기가 높아지는 추세∙AI21 역시 맘바 아키텍처를 토대로 더 빠른 추론 시간과 더 긴 컨텍스트를 지원하는 잠바 아키텍처를 활용해 기반모델을 개발n고센 CEO

질문을 입력하세요 (종료하려면 'exit' 입력):  AI21의 CEO에 대해 설명해줘



Prompt 1: prompt1.txt
Debug Output: {'context': [Document(metadata={'source': 'ai_industry.pdf', 'page': 18}, page_content='SPRi AI Brief |  2024-11월호\n16\nAI21 CEO, AI 에이전트에 트랜스포머 아키텍처의 대안 필요성 강조n이스라엘 AI 스타트업 AI21의 오리 고센 CEO는 AI 모델 개발에 주로 활용되는 트랜스포머 아키텍처가 느린 속도와 과도한 연산 비용으로 인해 AI 에이전트에 부적합하다고 지적n고센 CEO는 AI 에이전트를 활성화하려면 메모리 사용을 최적화하여 효율적 연산과 비용 절감을 지원하는 맘바나 잠바와 같은 대체 아키텍처에 주목해야 한다고 주장   \nKEY Contents'), Document(metadata={'source': 'ai_industry.pdf', 'page': 4}, page_content='SPRi AI Brief |  2024-11월호\n2\n미국 백악관 예산관리국, 정부의 책임 있는 AI 조달을 위한 지침 발표n미국 백악관 예산관리국이 바이든 대통령의 AI 행정명령에 따라 연방정부의 책임 있는 AI 조달을 지원하기 위한 지침을 발표 n지침은 정부 기관의 AI 조달 시 AI의 위험과 성과를 관리할 수 있는 모범 관행의 수립 및 최상의 AI 솔루션을 사용하기 위한 공급업체 시장의 경쟁 보장, 정부 기관 간 협업을 요구  \nKEY Contents'), Document(metadata={'source': 'ai_industry.pdf', 'page': 18}, page_content='£AI 에이전트 활성화를 위해 향상된 메모리 성능을 갖춘 대체 아키텍처 채택 필요n이스라엘의 AI 스타트업 AI21의 오리 고센(Ori Goshen) CEO가 AI 에이전트를 활성화하려면 트랜스포머(Transformer)* 이외의 새로운 아키텍처**가 필요하다고 주장* 문장 속 단어와 같은 순차 데이터 내의 관계를 추적

질문을 입력하세요 (종료하려면 'exit' 입력):  gira는 누가 언제 발표했지?


...

질문을 입력하세요 (종료하려면 'exit' 입력):  맘바



Prompt 1: prompt1.txt
Debug Output: {'context': [Document(metadata={'source': 'ai_industry.pdf', 'page': 10}, page_content='£라마 3.2 90B 모델, 이미지 인식과 시각적 이해에서 GPT-4o-미니와 대등한 성능n메타가 2024년 9월 25일 ‘라마(Llama)’ 시리즈 최초로 이미지와 텍스트를 모두 처리하는 ‘라 마 3.2’를 공개∙라마 3.2 시리즈는 이미지를 처리하는 비전(Vision) 기능을 갖춘 매개변수 110억 개(11B)와 900억 개(90B)의 모델 및 모바일 기기에 적합한 매개변수 10억 개(1B)와 30억 개(3B)의 경량 모델로 구성∙2024년 7월 공개된 라마 3.1과 비교해 라마 3.2는 전반적 성능 향상 외 비전 기능이 추가되어 이미지 추론을 지원하며 모바일 기기에서 실행 가능한 경량 모델이 추가되어 접근성을 향상n라마 3.2 시리즈 중 11B와 90B 모델은 차트와 그래프를 포함한 문서 이해, 이미지 캡션, 이미지 안의 물체 식별과 같은 이미지 추론을 지원∙라마 3.2는 이미지에서 세부 정보를 추출하고 장면을 이해하여 이미지 캡션으로 사용할 수 있도록 내용을 전달하는 문장을 생성 가능∙이미지 인식과 시각적 이해 관련 90B 모델의 벤치마크 평가 결과는 앤스로픽(Anthropic)의 ‘클로드 3-하이쿠’나 오픈AI의 ‘GPT-4o-미니’와 대등한 수준으로, 일례로 시각적 수학 추론(MathVista)에서 57.3점으로 클로드 3-하이쿠(46.4점)와 GPT-4o-미니(56.7점)를 능가 n라마 3.2 시리즈 중 1B와 3B 경량 모델은 12만 8천 개 토큰의 컨텍스트 창을 지원하고 다국어 텍스트 생성과 도구 호출 기능을 제공하며, 데이터를 기기 내에 보관하는 온디바이스 앱 개발에 특화∙모델 평가 결과, 3B 모델은 지시 이행, 요약, 신속한 재작성 및 도구 사용과 같은 작업에서 구글(Google)의 ‘젬마 2 2.6B’ 및 마이크로소프

질문을 입력하세요 (종료하려면 'exit' 입력):  SPRi에 대해 알려줘



Prompt 1: prompt1.txt
Debug Output: {'context': [Document(metadata={'source': 'ai_industry.pdf', 'page': 24}, page_content='경기도 성남시 분당구 대왕판교로 712번길 22 글로벌 R&D 연구동(B) 4층22, Daewangpangyo-ro 712beon-gil, Bundang-gu, Seongnam-si, Gyeonggi-do, Republic of Korea, 13488\n홈페이지 : https://spri.kr/보고서와 관련된 문의는 AI정책연구실(hs.lee@spri.kr, 031-739-7333)로 연락주시기 바랍니다.'), Document(metadata={'source': 'ai_industry.pdf', 'page': 18}, page_content='SPRi AI Brief |  2024-11월호\n16\nAI21 CEO, AI 에이전트에 트랜스포머 아키텍처의 대안 필요성 강조n이스라엘 AI 스타트업 AI21의 오리 고센 CEO는 AI 모델 개발에 주로 활용되는 트랜스포머 아키텍처가 느린 속도와 과도한 연산 비용으로 인해 AI 에이전트에 부적합하다고 지적n고센 CEO는 AI 에이전트를 활성화하려면 메모리 사용을 최적화하여 효율적 연산과 비용 절감을 지원하는 맘바나 잠바와 같은 대체 아키텍처에 주목해야 한다고 주장   \nKEY Contents'), Document(metadata={'source': 'ai_industry.pdf', 'page': 4}, page_content='SPRi AI Brief |  2024-11월호\n2\n미국 백악관 예산관리국, 정부의 책임 있는 AI 조달을 위한 지침 발표n미국 백악관 예산관리국이 바이든 대통령의 AI 행정명령에 따라 연방정부의 책임 있는 AI 조달을 지원하기 위한 지침을 발표 n지침은 정부 기관의 AI 조달 시 AI의 위험과 성과를 관리할 수 있는 모범 관행의 수립 및 최상의 AI 솔루션을 사용하기 

질문을 입력하세요 (종료하려면 'exit' 입력):  exit


# 평가기준
- 평가항목은 각각 1점부터 5점까지로 평가되며 1점은 전혀 만족하지 못 함, 5점은 매우 만족함
- **정확성:** 응답이 문서 내용에 근거하여 올바른지
- **간결성:** 응답이 지나치게 장황하고 불필요하지 않은지
  - prompt1에서 문서에 포함되지 않았다는 답변은 간결성을 평가하지 않음
  - prompt2는 100자 제한을 만족했는지 확인
- **가독성:** 응답이 읽기 쉬운지
- **목표달성도:** 프롬프트가 의도한 목표를 잘 달성했는지
  - prompt1: 문서 기반 정확한 응답
  - prompt2: 문서 기반 간결한 응답
  - prompt3: 문서 기반 쉽고 친절한 응답 및 기관 홍보

# 점수
| Prompt   | 정확성 (40%) | 간결성 (10%) | 가독성 (10%) | 목표달성도 (40%) | 총점    |
|----------|--------------|--------------|--------------|------------------|---------|
| prompt1  | 3.3          | 4.3          | 5            | 3.3              | 3.57 /5 |
| prompt2  | 4.1          | 4.7          | 4.8          | 4.2              | 4.27 /5 |
| prompt3  | 4.2          | 3.7          | 4.9          | 4.6              | 4.38 /5 |

# 평가결과

## prompt1
1. 프롬프트 엔지니어링 전 코드로 문맥에 맞게 대답하라고 지시한 모델보다도 여러 방면에서 더 나쁜 성능을 보이는 듯함
2. 문서의 내용에 따라 답변하도록 강조하여 오히려 유연성이 떨어지는 것으로 보임

## prompt2
1. 답변을 최대한 하라고 노력하라는 지침을 주자 오히려 적절한 답변들이 나올 때가 있음
2. 다만 질문이 부정확하거나 난해하면 잘못된 답변을 내놓게 됨

## prompt3
1. 주의사항 지침 3, 4번이 정확하게 작성되지 않은 문제로 간결성 점수가 낮아짐
2. 홍보담당이라는 역할을 준 덕분에 사용자의 편의가 증가함
3. 친절한 톤이지만 그 때문에 답변이 길어짐

# 결론 및 점검사항

- **prompt2**와 **prompt3**을 적절히 수정하고 결합하면 더 좋은 프롬프트가 될 것 같다.
- **평가기준의 객관성과 적절성**을 확보할 필요가 있다.